## WideNet prototype

In [1]:
%run Setup.ipynb
%run ExtraFunctions.ipynb

[i] Loaded Parameters:
 40000 35 0.2 200 
 dataset/glove/glove.twitter.27B.200d.txt
[i] Importing Modules...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


[+] Using Keras version 2.1.4
[i] Finished Importing Modules
[i] . Reading from csv file...Done!
[i] Found 30505 unique tokens.
[+] Shape of data tensor: (40000, 30)
[+] Shape of label tensor: (40000, 6)
[+] Number of entries in each category:
[+] Training:
 [ 7706.  7417.  4811. 10909.  1066.    91.]
[+] Validation:
 [1938. 1853. 1218. 2715.  257.   19.]
[i] Loading GloVe from: dataset/glove/glove.twitter.27B.200d.txt ...Done.
[+] Proceeding with Embedding Matrix...Completed!
Finished running setup.


In [2]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Feed the input vector into the embedding layer, then feed the resulting sequence into a bidirectional LSTM layer.

In [3]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
l_lstm1 = Bidirectional(LSTM(4,return_sequences=True))(embedded_sequences)

In [7]:
inception = [] # append our layers in parallel

# kernel 7: best accuracy, some over fitting!
l_conv_7 = Conv1D(filters=12,kernel_size=7,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
# kernel 2: good accuracy
l_conv_2 = Conv1D(filters=12,kernel_size=2,activation='relu')(l_lstm1)
# kernel 6: good accuracy
l_conv_6 = Conv1D(filters=12,kernel_size=6,activation='relu')(l_lstm1)
# kernel 3: good accuracy, no over fitting
l_conv_3 = Conv1D(filters=12,kernel_size=3,activation='relu')(l_lstm1)

inception.append(l_conv_7)
inception.append(l_conv_2)
inception.append(l_conv_6)
inception.append(l_conv_3)

# poorer performing layers
l_conv_4 = Conv1D(filters=12,kernel_size=4,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
l_drop_4 = Dropout(0.6)(l_conv_4)
l_conv_5 = Conv1D(filters=12,kernel_size=5,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
l_drop_5 = Dropout(0.6)(l_conv_5)
l_conv_8 = Conv1D(filters=12,kernel_size=8,activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_lstm1)
l_drop_8 = Dropout(0.6)(l_conv_8)

inception.append(l_drop_4)
inception.append(l_drop_5)
inception.append(l_drop_8)

In [8]:
# last parallel layer in inception: max-pooling layer
l_pool_i1 = MaxPooling1D(3)(l_lstm1)
l_conv_1 = Conv1D(filters=12,kernel_size=1,
                    activation='relu',kernel_regularizer=regularizers.l2(0.01))(l_pool_i1)
inception.append(l_conv_1)

In [9]:
l_merge = Concatenate(axis=1)(inception)
l_pool = MaxPooling1D(3)(l_merge)
l_drop = Dropout(0.4)(l_pool)
l_flat = Flatten()(l_drop)
l_dense = Dense(16, activation='relu')(l_flat)
preds = Dense(6, activation='softmax')(l_dense)

In [10]:
model = Model(sequence_input, preds)
adadelta = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
lr_metric = get_lr_metric(adadelta)
model.compile(loss='categorical_crossentropy',
              optimizer=adadelta,
              metrics=['acc', lr_metric])

In [11]:
!rm -r logs

In [12]:
tensorboard = callbacks.TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=16, write_grads=True , write_graph=True)
model_checkpoints = callbacks.ModelCheckpoint("checkpoint.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
lr_schedule = callbacks.LearningRateScheduler(initial_boost)

In [13]:
model.summary()
model.save('WideNet.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 30)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 30, 200)      6101200     input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 30, 8)        6560        embedding_1[0][0]                
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 27, 12)       396         bidirectional_1[0][0]            
__________________________________________________________________________________________________
conv1d_14 

In [14]:
print("Training Progress:")
model_log = model.fit(x_train, y_train, validation_data=(x_val, y_val),
          epochs=200, batch_size=50,
          callbacks=[tensorboard, lr_schedule])

Training Progress:
Train on 32000 samples, validate on 8000 samples
Epoch 1/200
32000/32000 [==============================] - 20s 630us/step - loss: 1.3953 - acc: 0.4308 - lr: 6.0000 - val_loss: 1.2699 - val_acc: 0.4740 - val_lr: 6.0000
Epoch 2/200
32000/32000 [==============================] - 20s 634us/step - loss: 1.2581 - acc: 0.4905 - lr: 1.0000 - val_loss: 1.2495 - val_acc: 0.4914 - val_lr: 1.0000
Epoch 3/200
32000/32000 [==============================] - 20s 633us/step - loss: 1.2443 - acc: 0.4976 - lr: 1.0000 - val_loss: 1.2453 - val_acc: 0.5036 - val_lr: 1.0000
Epoch 4/200
32000/32000 [==============================] - 20s 625us/step - loss: 1.2335 - acc: 0.5040 - lr: 1.0000 - val_loss: 1.2363 - val_acc: 0.5006 - val_lr: 1.0000
Epoch 5/200
32000/32000 [==============================] - 20s 629us/step - loss: 1.2253 - acc: 0.5085 - lr: 1.0000 - val_loss: 1.2363 - val_acc: 0.5079 - val_lr: 1.0000
Epoch 6/200
32000/32000 [==============================] - 20s 611us/step - loss: 

KeyboardInterrupt: 

In [ ]:
import pandas as pd
model.save('WideNet.h5')
pd.DataFrame(model_log.history).to_csv("history-inception.csv")